<a href="https://colab.research.google.com/github/LukeSchmitt96/gym-pybullet-drones/blob/master/tether_sim/DQN_FR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/LukeSchmitt96/gym-pybullet-drones

Cloning into 'gym-pybullet-drones'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 2067 (delta 105), reused 51 (delta 29), pack-reused 1892
Receiving objects: 100% (2067/2067), 102.63 MiB | 43.77 MiB/s, done.
Resolving deltas: 100% (1276/1276), done.


In [2]:
pip install gym

In [3]:
pip install pybullet

     |████████████████████████████████| 87.5MB 34kB/s 


In [4]:
pip install stable-baselines3

     |████████████████████████████████| 153kB 7.9MB/s 


In [5]:
pip install 'ray[rllib]'

     |████████████████████████████████| 23.1MB 120kB/s 
     |████████████████████████████████| 133kB 52.2MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 204kB 59.5MB/s 
     |████████████████████████████████| 81kB 14.0MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 1.3MB 50.2MB/s 
     |████████████████████████████████| 2.9MB 52.3MB/s 
     |████████████████████████████████| 1.8MB 49.4MB/s 
     |████████████████████████████████| 36.4MB 88kB/s 
     |████████████████████████████████| 317kB 49.3MB/s 
     |████████████████████████████████| 61kB 10.7MB/s 
     |████████████████████████████████| 296kB 57.9MB/s 
     |████████████████████████████████| 143kB 62.1MB/s 
     |████████████████████████████████| 102kB 14.7MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl size=12622 sha256=5a8ca3c5a1785f219cf9a66e9d0659eab09dc6358cc4e69a42dc6b36ad032e34
  Stored in 

In [6]:
cd gym-pybullet-drones/

/content/gym-pybullet-drones


In [7]:
pip install -e .

Obtaining file:///content/gym-pybullet-drones
  Running setup.py develop for gym-pybullet-drones


In [ ]:
#!/usr/bin/env python

import keras, tensorflow as tf, numpy as np, gym, sys, copy, argparse
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import model_from_json
import collections
import matplotlib.pyplot as plt
import random

import numpy as np

from gym_pybullet_drones.envs.RLTetherAviary import RLTetherAviary
# from gym_pybullet_drones.envs.RLCrazyFlieAviary import RLCrazyFlieAviary

class DQN():

	def __init__(self, environment):
		
		self.environment = environment
		self.num_actions = self.environment.action_space.shape[0] # Pitch forward, Pitch backward, Roll left, Roll right, Hover
		self.state_size = self.environment.reset().shape[0]
		self.model = Sequential()
		self.model.add(Dense(64, input_dim=self.state_size, activation = 'relu')) # Define number of layers, neurons and activation
		self.model.add(Dense(64, activation = 'relu')) # Define number of layers, neurons and activation
		self.model.add(Dense(32, activation = 'relu')) # Define number of layers, neurons and activation
		self.model.add(Dense(self.num_actions))
		self.model.compile(loss = 'mse', optimizer = Adam(lr = 0.001)) # Define loss and learning rate

	def save_model_weights(self):
		
		model_json = self.model.to_json()
		with open("model.json", "w") as json_file:
			json_file.write(model_json)
		self.model.save_weights("model.h5")

	def load_model(self, model_file):
		
		json_file = open('model.json', 'r')
		loaded_model_json = json_file.read()
		json_file.close()
		self.model = model_from_json(loaded_model_json)

	def load_model_weights(self, weight_file):
		
		self.model.load_weights("model.h5")


class Replay_Memory():

	def __init__(self, memory_size = 50000): # Define replay memory size
		
	 	self.memory = collections.deque(maxlen = memory_size)

	def sample_batch(self, batch_size = 64): # Define batch size
		
		if len(self.memory) < batch_size:
			return None
		else:
			return random.sample(self.memory, batch_size)

	def append(self, transition):
		
		self.memory.append(transition)

class DQN_Agent():

	def __init__(self, environment_name):

		# self.environment_name = environment_name
		self.env = environment_name # gym.envs.make(self.environment_name)
		self.train_network = DQN(self.env)
		self.target_network = DQN(self.env)
		self.target_network.model.set_weights(self.train_network.model.get_weights())
		self.memory = Replay_Memory()
		self.num_episodes = 1500 # Define number of episodes
		self.gamma = 0.99 # Define discount rate
		self.eps_limit = 0.5 # Define Epsilon
		self.target_update = 5 # Define target net update frequency
		self.max_tether_force = self.env.MAX_TETHER_FORCE
		self.tether_length = self.env.TETHER_MIN_LENGTH

	def index_to_array(self, index, state):

		# Hover RPM = 14468 Max RPM = 21702 Tether Length = 0.2 Max Tether Force = 0.7144
		# Hover / Max = 0.66
		distance = np.linalg.norm(state[0:3])
		if distance > self.tether_length:
			tether_force = (np.absolute(distance - self.tether_length) / distance) # Define max tether force
		else:
			tether_force = 0 

		# Set 1
		min = 0.62 # Define RPM
		max = 0.70 # Define RPM

		# Set 2
		# min = 0.52 # Define RPM
		# max = 0.80 # Define RPM

		# Set 3
		# min = 0.42 # Define RPM
		# max = 0.90 # Define RPM
		
		hov = 0 # Define Hover RPM

		if index == 0:
			action_array = np.array([min, min, max, max, tether_force]) # Rotor 3 and Rotor 4, X-axis
		elif index == 1:
			action_array = np.array([max, max, min, min, tether_force]) # Rotor 1 and Rotor 2, X-axis
		elif index == 2:
			action_array = np.array([max, min, max, min, tether_force]) # Rotor 1 and Rotor 3, Y-axis
		elif index == 3:
			action_array = np.array([min, max, min, max, tether_force]) # Rotor 2 and Rotor 4, Y-axis
		elif index == 4:
			action_array = np.array([hov, hov, hov, hov, tether_force]) # Hover
		
		return action_array

	def epsilon_greedy_policy(self, q_values,state):
		
		eps = np.random.random(1)[0]
		if eps < self.eps_limit:
			action_index = np.random.randint(0, self.train_network.num_actions)
	
		else:
			action_index = np.argmax(q_values)
		
		action = self.index_to_array(action_index, state)

		current_q = q_values[action_index]
		
		return action_index, action, current_q

	def greedy_policy(self, q_values):
		
		action = np.argmax(q_values)

		return action
	
	def update_model(self):
	
		states = []
		obs_states = []
		batch = self.memory.sample_batch()

		if batch == None:
			return
		
		else:
			for tup in batch:
				state, action_index, reward, obs, done = tup
				states.append(state)
				obs_states.append(obs)
			
			states = np.asarray(states)
			obs_states = np.asarray(obs_states)

			target_q = (self.train_network.model.predict(states))
			target_q_obs = (self.target_network.model.predict(obs_states))
				
			i = 0	
			for tup in batch:
				state, action_index, reward, obs, done = tup
				if done:
					target_q[i][action_index] = reward
				else:
					q_s = max(target_q_obs[i])
					target_q[i][action_index] = reward + q_s * self.gamma
				i += 1

			self.train_network.model.train_on_batch(states, target_q) ##### Backprop qvalues

	def train(self):
		
		self.test_list = []
		self.train_td = []
		self.reward_plot = []
		episode = 0
		while episode < self.num_episodes:
			done = False
			state = self.env.reset()
			# print('Reset State', state, 'Shape', np.size(state))
			episode_td = []
			episode_reward = 0
			while not done:
				q_values = (self.train_network.model.predict(state[None, :]))[0]
				action_index, action, q = self.epsilon_greedy_policy(q_values, state)
				obs, reward, done,  _ = self.env.step(action)
				episode_reward += reward
				
				td_step = abs(reward + self.gamma * np.max((self.target_network.model.predict(obs[None, :]))) - q)
				episode_td.append(td_step)
				self.memory.append((state, action_index, reward, obs, done))
				self.update_model()
				state = obs

			self.train_td.append(np.mean(np.array(episode_td)))
			self.reward_plot.append(episode_reward)

			print('Episode: ', episode)
			print('Current Train Reward: ', episode_reward)
			print('Current TD Error: ', self.train_td[-1])
			print('\n')

			if episode % self.target_update == 0:
				self.target_network.model.set_weights(self.train_network.model.get_weights())
			
			if episode > 50:
				self.eps_limit = max(self.eps_limit - 0.005, 0.01) # Define exploration decay rate
			
			if episode % 100 == 0: 
				self.train_network.save_model_weights()
			
			if episode % 100 == 0: 
				current_test_reward = self.test(20)
				self.test_list.append(current_test_reward)
				print('Current Test Reward: ', current_test_reward)
				np.savetxt('td_error.txt', self.train_td)
				np.savetxt('test_list.txt', self.test_list)
				
				if current_test_reward >= 10000: 
					print('Problem solved!')
					self.train_network.save_model_weights()
					
					self.num_episodes = episode + 2

			episode += 1
		
		self.test_reward = self.test_list
		
		np.savetxt('td.txt', self.train_td)
		np.savetxt('reward.txt', self.test_reward)
		plt.figure(figsize = (12, 8))
		plt.plot(np.arange(len(self.test_reward)) * 100, np.ones(len(self.test_reward)) * 200, label = 'Target reward')
		plt.plot(np.arange(len(self.test_reward)) * 100, self.test_reward, label = 'Mean reward')
		plt.grid()
		plt.xlabel('Number of episodes', fontsize = 18)
		plt.ylabel('Cumulative reward', fontsize = 18)
		plt.legend(fontsize = 18)
		plt.show()
		
		plt.figure(figsize = (12, 8))
		plt.plot(np.arange(len(self.train_td)) * 100, self.train_td, label = 'Train TD Error')
		plt.grid()
		plt.xlabel('Number of Training Episodes', fontsize = 18)
		plt.ylabel('Average TD Error', fontsize = 18)
		plt.legend(fontsize = 18)
		plt.show()
	
		plt.figure(figsize = (12, 8))
		plt.plot(self.reward_plot, label = 'Train TD Error')
		plt.xlabel('Number of Training Episodes', fontsize = 18)
		plt.ylabel('Episode Rewards', fontsize = 18)
		plt.show()

	def test(self, episodes):
		
		test_reward = []
		for i in range(episodes):
			done = False
			state = self.env.reset()
			episode_reward = 0
			while not done:
				action_index = np.argmax(self.target_network.model.predict(state[None, :]))
				action = self.index_to_array(action_index, state)
				obs, reward, done, _ = self.env.step(action)
				episode_reward += reward
				state = obs
			test_reward.append(episode_reward)

		return np.mean(test_reward)

def main():

	MODEL = DQN_Agent(RLTetherAviary(gui = 0, record = 0))
	# MODEL.train_network.load_model('model.json')
	# MODEL.train_network.load_model_weights('model.h5')
	# MODEL.train_network.model.compile(loss = 'mse', optimizer = Adam(lr = 0.001))
	MODEL.train()
	
if __name__ == '__main__':
	main()

[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 0.000000, km 0.000000,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
plugin= 4
[INFO] Max tether force set to  0.7144 N


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Episode:  0
Current Train Reward:  -2058.625865130163
Current TD Error:  39.639249058658


Current Test Reward:  -2230.94538565244
Episode:  1
Current Train Reward:  -2328.1442183741983
Current TD Error:  41.264412959069354


Episode:  2
Current Train Reward:  -2386.470119762704
Current TD Error:  30.332787609571213


Episode:  3
Current Train Reward:  -1843.8517752701234
Current TD Error:  30.887071909010317


Episode:  4
Current Train Reward:  -2451.3027735432515
Current TD Error:  31.470630205832755


Episode:  5
Current Train Reward:  -2319.146617234302
Current TD Error:  36.79755030741922


